In [16]:
import yfinance as yf
import pandas as pd
from backtesting import Strategy, Backtest
from backtesting.test import SMA
from backtesting.lib import crossover, plot_heatmaps, resample_apply, barssince
import talib as ta


In [17]:
# Get stock data

stock = yf.download("AAPL", start="2023-01-01", end="2024-12-31")[
    ["Open", "High", "Low", "Close", "Volume"]
    ]
"""
he = yf.download("HE", start="2023-01-01", interval="15m")[
    ["Open", "High", "Low", "Close", "Volume"]
]
"""
stock.columns = stock.columns.droplevel(1)
#stock.head()
#stock.tail()

[*********************100%***********************]  1 of 1 completed


In [18]:
# Mean reversion / Bollinger Band strategy

def std_3(arr, n):
    return pd.Series(arr).rolling(n).std() * 3

class BB(Strategy):
    roll = 50
    def init(self):
        self.he = self.data.Close
        self.he_mean = self.I(SMA, self.he, self.roll)
        self.he_std = self.I(std_3, self.he, self.roll)
        self.he_upper = self.he_mean + self.he_std
        self.he_lower = self.he_mean - self.he_std
        self.he_close = self.I(SMA, self.he, 1)
    
    def next(self):
        if self.he_close < self.he_lower:
            self.buy(
                tp = self.he_mean,
            )
        if self.he_close > self.he_upper:
            self.sell(
                tp = self.he_mean,
            )

In [19]:
# Relative strength indicator strategy

class RSI(Strategy):
    rsi_period = 7
    rsi_upper_bound = 75
    rsi_lower_bound = 25
    
    def init(self):
        close = self.data.Close
        self.daily_rsi = self.I(ta.RSI, close, self.rsi_period)
        #self.weekly_rsi = resample_apply('W-FRI', ta.RSI, close, self.rsi_period)
        #self.position_size = 0.5
    
    def next(self):
        price = self.data.Close[-1]
        self.buy(
            sl = 0.95 * price
        )
        if crossover(self.rsi_lower_bound, self.daily_rsi):
        #    and self.weekly_rsi[-1] < self.rsi_lower_bound):
        #   and barssince(crossover(self.rsi_lower_bound, self.daily_rsi)) == 3:
            """
            if self.position.is_short:
                self.position.close()
            self.buy()
            """
            self.buy(
            #    size = self.position_size,
                sl=0.95*price,
            #    tp=1.2*price
                )
        elif crossover(self.daily_rsi, self.rsi_upper_bound):
        #      and self.weekly_rsi[-1] > self.rsi_upper_bound):
            #"""
            if self.position.is_long:
                self.position.close()
            #self.sell()
            #"""
            #self.position.close()

In [ ]:
bt = Backtest(stock, RSI, cash=10000, commission=.002)
stats = bt.run()
bt.plot()
print(stats)
print(stats._trades)

Start                     2023-01-03 00:00:00
End                       2024-12-30 00:00:00
Duration                    727 days 00:00:00
Exposure Time [%]                    97.80439
Equity Final [$]                  14159.09781
Equity Peak [$]                   14822.21491
Commissions [$]                    1677.59074
Return [%]                           41.59098
Buy & Hold Return [%]                91.02926
Return (Ann.) [%]                    19.11597
Volatility (Ann.) [%]                27.65436
CAGR [%]                             12.81152
Sharpe Ratio                          0.69125
Sortino Ratio                         1.16279
Calmar Ratio                          0.81236
Max. Drawdown [%]                   -23.53141
Avg. Drawdown [%]                    -3.43136
Max. Drawdown Duration      338 days 00:00:00
Avg. Drawdown Duration       28 days 00:00:00
# Trades                                   34
Win Rate [%]                         61.76471
Best Trade [%]                    